# API Call to get data from OpenStreetMap

In this notebook Data from the overpass API will be querryed to provide informations about vegan restaurtants in kiel. 

This Notebook is used to gather the data provited in the restaurants_kiel.csv file.

Date of API-Call: 2023-10-10

This was inspired by https://github.com/balzer82/OSM2Pandas/blob/master/OSM2Pandas-Kreuzungen.ipynb

In [27]:
import pandas as pd
import requests

In [28]:
# Selecting area arround Kiel
bbox = [10.020299,54.232977,10.24498,54.410641]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

In [34]:
# Tags to querry from OSM
tags = ['diet:vegan', 'vegan', 'vegetarian', 'cuisine=vegan', 'organic']

compactOverpassQLstring = '[out:json][timeout:60];('

# Iterate over vegan-related tags
for tag in tags:
    compactOverpassQLstring += 'node["%s"="yes"](%s,%s,%s,%s);' % (tag, minLat, minLon, maxLat, maxLon)

compactOverpassQLstring += ');out body;>;out skel qt;'


In [30]:

osmrequest = {'data': compactOverpassQLstring}
osmurl = 'http://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

In [31]:
# Assuming you have the 'osm' variable with your Overpass API results
osmdata = osm.json()  # Call the method to get the data

# Extract the 'elements' from the data
osmdata = osmdata['elements']

# Create an empty list to store the data
data = []

for dct in osmdata:
    if 'type' in dct and dct['type'] == 'node':
        # Check if the "diet:vegan" tag is present and set to "yes"
        if 'tags' in dct and 'diet:vegan' in dct['tags'] and dct['tags']['diet:vegan'] == 'yes':
            node_data = {
                'id': dct['id'],
                'type': dct['type'],
                'lat': dct['lat'],
                'lon': dct['lon']
            }

            # Extract other tags from the dictionary and add them to 'node_data'
            for key, val in dct['tags'].items():
                node_data[key] = val

            data.append(node_data)

# Create a DataFrame from the list of dictionaries
osmdf = pd.DataFrame(data)


In [35]:
# Saving the Data in a .csv file
osmdf.to_csv("data/restaurants_kiel.csv")